In [ ]:
# !python -m pip install -U tifffile[all]
# !pip list
# Install dependencies
!python -m pip install --upgrade pip
!pip install tensorflow

Could not fetch URL https://pypi.org/simple/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pip/ (Caused by SSLError(SSLError(1, '[SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1129)'))) - skipping


In [69]:
import numpy as np
import tensorflow as tf 
from tensorflow.keras import layers, Model
from osgeo import gdal
import json
import matplotlib.pyplot as plt
import tifffile as tiff
import os
import pandas as pd

In [70]:
def load_json_data(json_file):
    with open(json_file, 'r') as file: 
        data = json.load(file)
    return data 

data_s1 = load_json_data('C:/Users/48504/Desktop/JADSMaster/DeepLearning/DeepLearningProject/DeepLearning2024/S1list.json')
data_s2 = load_json_data('C:/Users/48504/Desktop/JADSMaster/DeepLearning/DeepLearningProject/DeepLearning2024/S2list.json')

In [71]:
# ## load images as we saw on kaggle
# img = tiff.imread('/home/pedro/Documents/JADS/DeepLearning/SEN12FLOOD/0066/S2_2019-02-04_B01.tif')
# img_array = np.array(img)
# plt.imshow(img)
# print(img_array.shape)            

In [72]:
### This cell is for read the tiff images using the tiff python library
### Images, filenames and flooding labels are set to be stored in arrays while reading the images
### Function takes a particular directory (A Flloding scenario and fetch for each label in the flooding json files)
### It automatically fetch for the number of the folder being processed 
### And retrieve both the name of the file and the flooding label. e.g from the sequence:
### 
###
### {"0063": {"1": {"date": "2019-02-04", "FLOODING": false, "FULL-DATA-COVERAGE": true, "filename": "S2_2019-02-04"}, "count": 1, "folder": "0063", "geo": {"type": "Polygon", "coordinates": [[[28.29722, -15.382762], [28.297507, -15.429039], [28.345216, -15.428755], [28.344918, -15.382479], [28.29722, -15.382762]]]}},
### Based on the name of the folder "0063" it reads the images of the folder and it appends both filename and flooding label.
### To cross and validate whether there is flooding or not the folder "0200" is used on the next cell. 
### This is the second sequence in the S2 json data. Easy to validate

In [73]:
import tifffile as tiff
from sklearn.model_selection import train_test_split
from skimage.transform import resize

def load_images_from_directory(directory, s1_data, s2_data):
    images = []
    filenames = []
    labels = []
    
    folder_name = os.path.basename(directory)
    print(f"Processing folder: {folder_name}")

    for file_name in os.listdir(directory):
        if file_name.endswith('.tif') or file_name.endswith('.tiff'):
            json_data = s1_data if file_name.startswith('S1') else s2_data if file_name.startswith('S2') else None
            if json_data is None: 
                print(f"Skipping file (not S1 or S2): {file_name}")
                continue
            
            found = False
            for item in json_data.get(folder_name, {}).values():
                if isinstance(item, dict) and 'filename' in item and item['filename'] in file_name:
                    file_path = os.path.join(directory, file_name)
                    img = tiff.imread(file_path)
                    img_array = np.array(img)
                    images.append(img_array)
                    filenames.append(file_name)
                    labels.append(item.get('FLOODING', False))
                    found = True 
                    break
    return images, filenames, labels

In [74]:
### Following code is to get a list of all the flooding events (folders)
### present on the daataset. There is also code to select which folders are
### going to be selected to train and test the model

In [75]:
#### This code is to apply recursively the function to read the images to all 
### The folders present on the dataset. As the number of the sequence event is stored in a 
### dictionary everythins is stored on the same dictionary with sequence and flooding label
def process_folders(folder_list, main_directory, s1_data, s2_data):
    all_images = []
    all_labels = []
    scenario_labels = []  # Additional label for the scenario
    all_filenames = []

    for folder in folder_list:
        directory = os.path.join(main_directory, folder)
        images, filenames, labels = load_images_from_directory(directory, s1_data, s2_data)

        all_images.extend(images)
        all_labels.extend(labels)
        all_filenames.extend(filenames)
#         scenario_labels.extend([folder] * len(labels))  # Assign the folder name as the scenario label

    return all_images, all_labels, all_filenames

In [76]:
def list_folders(main_directory):
    return [f for f in os.listdir(main_directory) if os.path.isdir(os.path.join(main_directory, f))]

# Main SEN12FLOOD directory, whole data is there
main_directory = 'C:/Users/48504/Desktop/JADSMaster/DeepLearning/DeepLearningProject/DeepLearning2024'

# List all folders
all_folders = list_folders(main_directory)

# Manually select folders for training and testing
# test_folders = [str(i) for i in range(68)]  # Example folders for training
# train_folders = ['0001', '0002', '0003', '0004', '0005', '0006', '0007', '0008', '0009', '0010', '0011', '0012', '0013', '0014', '0015', '0016', '0018', '0020', '0021', '0022', '0023', '0024', '0025', '0026', '0027', '0028', '0029', '0030', '0031', '0033', '0034', '0035', '0036', '0037', '0042', '0043', '0044', '0045', '0046', '0047', '0048', '0050', '0053', '0054', '0055', '0057', '0059', '0060', '0061', '0063', '0065', '0066', '0067', '0068', '0069', '0070', '0071', '0072', '0073', '0074', '0075', '0076', '0077', '0079', '0080', '0081', '0082', '0084', '0085', '0086', '0088', '0089', '0090', '0091', '0093', '0094', '0095', '0096', '0097', '0098', '0099', '0100', '0101', '0102', '0103', '0104', '0105', '0106', '0107', '0108', '0109', '0111', '0115', '0116', '0117', '0118', '0120', '0121', '0122', '0123', '0124', '0125', '0126', '0127', '0128', '0130', '0131', '0132', '0133', '0134', '0135', '0137', '0138', '0139', '0140', '0141', '0143', '0144', '0145', '0146', '0147', '0148', '0149', '0150', '0151', '0154', '0155', '0156', '0157', '0158', '0159', '0160', '0161', '0162', '0163', '0165', '0166', '0167', '0168', '0169', '0170', '0171', '0173', '0174', '0176', '0177', '0178', '0181', '0182', '0184', '0186', '0187', '0188', '0191', '0192', '0193', '0194', '0196', '0198', '0199', '0200', '0201', '0203', '0204', '0205', '0206', '0207', '0208', '0209', '0210', '0212', '0213', '0214', '0215', '0216', '0217', '0218', '0219', '0220', '0221', '0222', '0223', '0225', '0226', '0227', ' 0229', '0230', '0231', '0232', '0233', '0234', '0235', '0236', '0238', '0240', '0241', '0243', '0244', '0245', '0246', '0247', '0248', '0249', '0250', '0253', '0254', '0255', '0256', '0257', '0258', '0259', '0260', '0261', '0262', '0263', '0266', '0267', '0271', '0272', '0273', '0274', '0275', '0276', '0277', '0278', '0279', '0280', '0281', '0282', '0285', '0286', '0287', '0288', '0290', '0293', '0294', '0295', '0296', '0298', '0299', '0300', '0301', '0303', ' 0304', '0305', '0306', '0307', '0308', '0309', '0310', '0311', '0313', '0316', '0318', '0319', '0320', '0321', '0323', '0324', '0325', '0326', '0327', '0328', '0329', '0330', '0331', '0332', '0333', '0334', '0335', '0336']   # Example folders for testing
train_folders = ['0200']
test_folders = ['26']
validation_folders = ['61']


In [77]:
# KEEPNG THE CODE BELOW. DISCUSS IT WITH PEDRO

In [78]:
# ## Preprocess train and test folders
# train_images, train_labels, train_scenario_labels = process_folders(train_folders, main_directory, data_s1, data_s2)
# test_images, test_labels, test_scenario_labels = process_folders(test_folders, main_directory, data_s1, data_s2)
# validation_images, validation_labels, validation_scenario_labels = process_folders(validation_folders, main_directory, data_s1, data_s2)

In [79]:
### Preprocess train and test folders
train_images, train_labels, train_filenames = process_folders(train_folders, main_directory, data_s1, data_s2)
test_images, test_labels, test_filenames = process_folders(test_folders, main_directory, data_s1, data_s2)
validation_images, validation_labels, validation_filenames = process_folders(validation_folders, main_directory, data_s1, data_s2)

Processing folder: 0200
Processing folder: 26
Processing folder: 61


In [80]:
# There is one missing thing here.
# We can parse mutiple folders, and collect the images and labels. However, we are still missing to keep track individual files
# That is important later for printing the images. So the 

In [81]:
# # Here we can predefine training, test and validation 
for filename, label in zip(validation_filenames, validation_labels):
    print(f"Filename validation: {filename}, Flooding: {label}")

for filename, label in zip(train_filenames, train_labels):
    print(f"Filename train: {filename}, Flooding: {label}")

for filename, label in zip(test_filenames, test_labels):
    print(f"Filename test: {filename}, Flooding: {label}")

# # Assuming images is your list of numpy arrays representing images
resized_validation_images = []
resized_test_images = []
resized_train_images = []


# Define the target shape
target_shape = (522, 544)  

from skimage.transform import resize

# Define the target shape
target_shape = (522, 544)  # Example target shape, adjust as needed

resized_train_images = []
for image in train_images:
    resized_image = resize(image, target_shape, preserve_range=True)
    resized_train_images.append(resized_image)

resized_test_images = []
for image in test_images:
    resized_image = resize(image, target_shape, preserve_range=True)
    resized_test_images.append(resized_image)

resized_validation_images = []
for image in validation_images:
    resized_image = resize(image, target_shape, preserve_range=True)
    resized_validation_images.append(resized_image)

# Convert the resized images and labels into numpy arrays
X_train = np.array(resized_train_images)
X_test = np.array(resized_test_images)
X_val = np.array(resized_validation_images)
y_train = np.array(train_labels)
y_test = np.array(test_labels)
y_val = np.array(validation_labels)

Filename validation: S1A_IW_GRDH_1SDV_20190223T031705_20190223T031730_026051_02E795_2CA5_corrected_VH.tif, Flooding: False
Filename validation: S1A_IW_GRDH_1SDV_20190223T031705_20190223T031730_026051_02E795_2CA5_corrected_VV.tif, Flooding: False
Filename validation: S1A_IW_GRDH_1SDV_20190307T031705_20190307T031730_026226_02EDD4_6209_corrected_VH.tif, Flooding: False
Filename validation: S1A_IW_GRDH_1SDV_20190307T031705_20190307T031730_026226_02EDD4_6209_corrected_VV.tif, Flooding: False
Filename validation: S1A_IW_GRDH_1SDV_20190318T162351_20190318T162420_026394_02F415_5335_corrected_VH.tif, Flooding: True
Filename validation: S1A_IW_GRDH_1SDV_20190318T162351_20190318T162420_026394_02F415_5335_corrected_VV.tif, Flooding: True
Filename validation: S1A_IW_GRDH_1SDV_20190319T031705_20190319T031730_026401_02F44F_2D37_corrected_VH.tif, Flooding: True
Filename validation: S1A_IW_GRDH_1SDV_20190319T031705_20190319T031730_026401_02F44F_2D37_corrected_VV.tif, Flooding: True
Filename validation:

In [ ]:
# import tenserflow as tensorflow
from tensorflow.keras import layers, models

# Define the model architecture
model = models.Sequential([
    # Convolutional layers
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(522, 544, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    # Flatten layer to transition from convolutional layers to fully connected layers
    layers.Flatten(),
    # Fully connected layers
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Output layer with single neuron for binary classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Use binary cross-entropy loss for binary classification
              metrics=['accuracy'])  # Use accuracy as the metric for evaluation

# Reshape the input data to fit the model input shape
X_train = X_train.reshape((-1, 522, 544, 1))
X_test = X_test.reshape((-1, 522, 544, 1))
X_val = X_val.reshape((-1, 522, 544, 1))
                       
# Train the model
model.fit(X_train, y_train, epochs=1, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Predict labels for test data
y_pred = model.predict(X_test)
# Convert predicted probabilities to binary labels
y_pred_labels = (y_pred > 0.5).astype(int)

# Reshape predicted labels to match original image shape
# X_test_original_shape = X_test.reshape((-1, 522, 544))

# Compare predicted labels with actual labels
for i in range(len(X_test)):
    print(f"Filename test: {test_filenames[i]} Actual: {y_test[i]}, Predicted: {y_pred_labels[i]}")

# Evaluate the model
train_loss, train_accuracy = model.evaluate(X_train, y_train)
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print('Train loss:', train_loss)
print('Test loss:', test_loss)

4/4 ━━━━━━━━━━━━━━━━━━━━ 104s 23s/step - accuracy: 0.5738 - loss: 30673.8652 - val_accuracy: 0.5797 - val_loss: 478.4587
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 3s/step
Filename test: S1B_IW_GRDH_1SDV_20181230T162345_20181230T162410_014273_01A8CE_D597_corrected_VH.tif Actual: False, Predicted: [1]
Filename test: S1B_IW_GRDH_1SDV_20181230T162345_20181230T162410_014273_01A8CE_D597_corrected_VV.tif Actual: False, Predicted: [0]
Filename test: S1B_IW_GRDH_1SDV_20190111T162345_20190111T162410_014448_01AE74_EF21_corrected_VH.tif Actual: False, Predicted: [0]
Filename test: S1B_IW_GRDH_1SDV_20190111T162345_20190111T162410_014448_01AE74_EF21_corrected_VV.tif Actual: False, Predicted: [0]
Filename test: S1B_IW_GRDH_1SDV_20190123T162344_20190123T162409_014623_01B40F_3BD7_corrected_VH.tif Actual: False, Predicted: [1]
Filename test: S1B_IW_GRDH_1SDV_20190123T162344_20190123T162409_014623_01B40F_3BD7_corrected_VV.tif Actual: False, Predicted: [0]
Filename test: S2_2018-12-28_B01.tif Actual: False, Predicted: 

In [19]:
## So the model seems to work, it is not the best of course but it works
## Now we need to plot these graphs as Matteo showed during one of his lectures

NameError: name 'img' is not defined